In [1]:
import pandas as pd
from datetime import datetime
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

##### Crashes data preprocessing 

In [2]:
crashes = pd.read_csv('data/car_crashes.csv').iloc[1:]

/Users/natali/anaconda3/envs/py37/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Some of the rows in CRASH_DATE column have no time, the folowing command adds 09:45 to these rows

In [3]:
crashes['CRASH_DATE'] = crashes['CRASH_DATE'].apply(lambda x: x if len(x) == 23 else x + ' ' + '09:45:00 PM')

To datetime format

In [4]:
crashes['CRASH_DATE'] = crashes['CRASH_DATE'].apply(lambda x: datetime.strptime(x, '%Y %b %d %I:%M:%S %p'))

Extracting only potentially important features 

In [5]:
crashes = crashes[['CRASH_DATE', 'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'ROADWAY_SURFACE_COND', 'INJURIES_TOTAL', 'INJURIES_FATAL', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH']];

Creating new feature consisting of date and hour of crash

In [6]:
crashes['DATE_HOUR'] = crashes['CRASH_DATE'].apply(lambda x : str(x.date()) + ' ' + str(x.hour))

##### Weather data preprocessing 

In [7]:
weather = pd.read_csv('data/2018-05-08.csv').iloc[:,1:]

In [8]:
weather.columns = ['Time', 'Temperature', 'Dew Point', 'Humidity', 'Wind', 'Wind Speed', 'Wind Gust', 'Pressure', 'Precip.', 'Precip Accum', 'Condition', "Date"]

In [9]:
weather['Time'] = weather['Time'].apply(lambda x: datetime.strptime(x, '%I:%M %p').hour)

In [10]:
weather['DATE_HOUR'] = weather['Date'].map(str).apply(lambda x: x.split(' ')[0]) + ' ' + weather["Time"].map(str)

In [11]:
for col_name in ['Temperature', 'Dew Point', 'Humidity', 'Wind Speed', 'Wind Gust', 'Pressure']:
    weather[col_name] = weather[col_name].map(str).apply(lambda x: x.split('\n')[0])

In [12]:
weather = weather.drop(['Precip.', 'Precip Accum', 'Condition', 'Wind'], axis = 1);

##### Merging two data sets

In [13]:
data_all = pd.merge(weather, crashes, on="DATE_HOUR").dropna().drop('Time', axis = 1)

In [14]:
data_all.to_csv('data/data_all.csv')